In [1]:
%config IPCompleter.greedy=True
import matplotlib.pyplot as plt
import numpy as np
from NuMPI.Tools.Reduction import Reduction
# from ContactMechanics import FreeFFTElasticHalfSpace
from SurfaceTopography import make_sphere
from ContactMechanics.ReferenceSolutions import Hertz as Hz

import scipy.optimize as optim
from math import isnan, sqrt

In [10]:
# sphere radius:
r_s = 3.0
# equivalent Young's modulus
E_s = 1.
nx, ny = (16,16)
#for disp0, normal_force in [(0.1, None), (0, 15.0)]:
disp0 = None
normal_force = 15.
sx = 6.0

substrate = FreeFFTElasticHalfSpace((nx, ny), E_s, (sx, sx))
                                    #fft='mpi',
                                    #communicator=comm)

#print(substrate.)
surface = make_sphere(
    r_s, (nx, ny), (sx, sx),
    nb_subdomain_grid_pts=substrate.topography_nb_subdomain_grid_pts,
    subdomain_locations=substrate.topography_subdomain_locations,
    kind="paraboloid")
    #communicator=substrate.communicator)
# system = NonSmoothContactSystem(substrate, surface)
# substrate._compute_greens_function
# print(np.fft.irfft(substrate.greens_function.real))
result = constrained_conjugate_gradients(substrate=substrate, topography=surface, offset=disp0,
                                external_force=15.0)


(6.0, 6.0) (16, 16)
(6.0, 6.0) (16, 16)
(6.0, 6.0) (16, 16)
(6.0, 6.0) (16, 16)
(6.0, 6.0) (16, 16)
(6.0, 6.0) (16, 16)
(6.0, 6.0) (16, 16)
(6.0, 6.0) (16, 16)
(6.0, 6.0) (16, 16)
(6.0, 6.0) (16, 16)
(6.0, 6.0) (16, 16)
(6.0, 6.0) (16, 16)
(6.0, 6.0) (16, 16)
(6.0, 6.0) (16, 16)
(6.0, 6.0) (16, 16)
(6.0, 6.0) (16, 16)
(6.0, 6.0) (16, 16)
(6.0, 6.0) (16, 16)
(6.0, 6.0) (16, 16)
(6.0, 6.0) (16, 16)
(6.0, 6.0) (16, 16)
(6.0, 6.0) (16, 16)
(6.0, 6.0) (16, 16)
(6.0, 6.0) (16, 16)
(6.0, 6.0) (16, 16)
(6.0, 6.0) (16, 16)
(6.0, 6.0) (16, 16)
[[-0.         -0.         -0.         -0.         -0.          0.01134241
   0.02577436  0.03278054  0.035066    0.0328228   0.02586478  0.01154409
  -0.         -0.         -0.         -0.        ]
 [-0.         -0.         -0.          0.00778272  0.0340223   0.04622795
   0.05253337  0.05583116  0.0569084   0.05585675  0.05259016  0.04633787
   0.0342406   0.00824363 -0.         -0.        ]
 [-0.         -0.          0.01440116  0.04182684  0.05362565 

In [ ]:
"""disp = result.x
forces = -result.jac
converged = result.success
assert converged
#result.x.shape
fig, ax = plt.subplots()
surf = ax.imshow(result.x[:512,:512])
plt.colorbar(surf, shrink=0.6, aspect=6)"""

In [9]:
def constrained_conjugate_gradients(substrate, topography, hardness=None,
                                    external_force=None, offset=None,
                                    initial_displacements=None,
                                    initial_forces=None,
                                    pentol=None,
                                    forcetol=1e-5,
                                    thermotol=1e-6,
                                    mixfac=0.1,
                                    mixdecfac=0.95,
                                    minmixsteps=10,
                                    #maxiter=3,
                                    maxiter=100000,
                                    callback=None,
                                    verbose=False):
    """
    Use a constrained conjugate gradient optimization to find the equilibrium
    configuration deflection of an elastic manifold. The conjugate gradient
    iteration is reset using the steepest descent direction whenever the
    contact area changes.

    This method is described in I.A. Polonsky, L.M. Keer, Wear 231, 206 (1999)

    For plastic calculations, the solver may switch to a simple overrelaxation.

    Parameters
    ----------
    substrate : elastic manifold
        Elastic manifold.
    topography : SurfaceTopography object
        Height profile of the rigid counterbody
    hardness : array_like, optional
        Hardness of the substrate. Force cannot exceed this value. Can be
        scalar or array (i.e. per pixel) value.
    external_force : float, optional
        External force. Constrains the sum of forces to this value.
    offset : float, optional
        Offset of rigid surface. Ignore if external_force is specified.
    initial_displacements : array_like, optional
        Displacement field for initializing the solver. Guess an initial
        value if set to None.
    initial_forces : array_like, optional
        pixel forces field for initializing the solver. Is computed from
        initial_displacements if none
    pentol : float, optional
        Maximum penetration of contacting regions required for convergence.
    forcetol : float, optional
        Maximum force outside the contact region allowed for convergence.
    thermotol : float, optional
        Maximum *relative* change in thermodynamic control property (total
        force at constant displacement or displacement at constant force)
        during convergence.
    mixfac : float, optional
        Mixing factor for simple overrelaxaton (only used for certain plastic
        calculations). (Default: 0.1)
    mixdecfac : float, optional
        Mixing factor is multiplied with this factor after every step.
        (Default: 0.9)
    minmixsteps : int, optional
        Minimum number of mixing steps. (Default: 10)
    maxiter : float, optional
        Maximum number of iterations.
    logger : :obj:`ContactMechanics.Tools.Logger`, optional
        Reports status and values at each iteration.
    callback : callable(int iteration, array_link forces, dict d), optional
        Called each iteration. The dictionary contains additional scalars.
    verbose : bool, optional
        If True, more scalar quantities are passed to the logger.

    Returns
    -------
    Optimisation result
        x: displacements
        fun: elastic energy
        jac: forces
        active_set: points where forces are not constrained to 0 or hardness
        offset: offset i rigid surface, results from the optimization processes
           when the external_force is constrained
    """
    if substrate.nb_subdomain_grid_pts != substrate.nb_domain_grid_pts:
        # check that a topography instance is provided and not only a numpy
        # array
        if not hasattr(topography, "nb_grid_pts"):
            raise ValueError("You should provide a topography object when working with MPI")

    reduction = Reduction(substrate.communicator)

    # surface is the array holding the data assigned to the processsor
    if not hasattr(topography, "nb_grid_pts"):
        heights = topography
        topography = Topography(heights, physical_sizes=substrate.physical_sizes)
    else:
        heights = topography.heights()  # Local data

    # Note: Suffix _r denotes real-space _q reciprocal space 2d-arrays

    nb_surface_pts = np.prod(topography.nb_grid_pts)
    if pentol is None:
        # Heuristics for the possible tolerance on penetration.
        # This is necessary because numbers can vary greatly
        # depending on the system of units.
        pentol = topography.rms_height_from_area() / (10 * np.mean(topography.nb_grid_pts))
        # If pentol is zero, then this is a flat surface. This only makes
        # sense for nonperiodic calculations, i.e. it is a punch. Then
        # use the offset to determine the tolerance
        if pentol == 0:
            pentol = (offset + reduction.sum(heights[...]) / nb_surface_pts) / 1000
        # If we are still zero use an arbitrary value
        if pentol == 0:
            pentol = 1e-3

    #logger.pr(f'maxiter = {maxiter}')
    #logger.pr(f'pentol = {pentol}')

    if offset is None:
        offset = 0

    if initial_displacements is None:
        u_r = np.zeros(substrate.nb_subdomain_grid_pts)
    else:
        u_r = initial_displacements.copy()
    #A_contact It's the same for PeriodicFFTElHS
    comp_slice = [slice(0, max(0, min(substrate.nb_grid_pts[i] - substrate.subdomain_locations[i],
                                      substrate.nb_subdomain_grid_pts[i])))
                  for i in range(substrate.dim)]
    if substrate.dim not in (1, 2):
        raise Exception(f'Constrained conjugate gradient currently only implemented for 1 or 2 dimensions (Your '
                        f'substrate has {substrate.dim}.).')

    comp_mask = np.zeros(substrate.nb_subdomain_grid_pts, dtype=bool)
    comp_mask[tuple(comp_slice)] = True

    surf_mask = np.ma.getmask(heights)
    if surf_mask is np.ma.nomask:
        surf_mask = np.ones(topography.nb_subdomain_grid_pts, dtype=bool)
    else:
        comp_mask[tuple(comp_slice)][surf_mask] = False
        surf_mask = np.logical_not(surf_mask)

    masked_surface = np.asarray(heights[surf_mask])
    max_masked_surface = reduction.max(masked_surface)

    # np.savetxt('heights.out', topography.heights(), delimiter='\t')
    
    pad_mask = np.logical_not(comp_mask)
    N_pad = reduction.sum(pad_mask * 1)
    u_r[comp_mask] = np.where(u_r[comp_mask] < masked_surface + offset, masked_surface + offset, u_r[comp_mask])

    # initialize result array 
    result = optim.OptimizeResult()
    result.nfev = 0
    result.nit = 0
    result.success = False
    result.message = "Not Converged (yet)"
 
    # print(topography.pixel_size[0])
    # Compute forces
    # f_r = -np.fft.ifft2(np.fft.fft2(u_r)/gf_q).real
    if initial_forces is None:
        f_r = substrate.evaluate_force(u_r)
    else:
        f_r = initial_forces.copy()
        u_r = substrate.evaluate_disp(f_r)

    result.nfev += 1
    # Force outside the computational region must be zero
    f_r[pad_mask] = 0.0

    # iteration
    delta = 0
    delta_str = 'reset'
    G_old = 1.0
    t_r = np.zeros_like(u_r)

    tau = 0.0

    current_mixfac = mixfac
    mixsteps = 0

    last_max_height = None
    last_total_force = None
        #fig, (ax1,ax2) = plt.subplots(2)
        #surf1 = ax1.imshow(f_r)
        #surf2 = ax2.imshow(u_r)
        #plt.colorbar(surf1)
        #plt.colorbar(surf2)

    for it in range(1, maxiter + 1):
        result.nit = it
        
        # Reset contact area (area that feels compressive stress)
        c_r = f_r < 0.0

        # Compute total contact area (area with compressive force)
        A_contact = reduction.sum(c_r * 1)

        # Compute total are treated by the CG optimizer (which exclude flowing
        # portions).
        A_cg = reduction.sum(c_r * 1)

        # We now decide if we do a Polonsky-Keer (sd/cg) step or a mixing step.
        # The Polonsky-Keer step is only possible if we have a "cg" controlled
        # area, i.e. an area where the pressure is between 0 and the hardness.
        # We here use the heuristic criterion that this should be more than 1%
        # of the overall contact area. In all other cases we mix.
        if hardness is None or (A_contact > 0 and A_cg / A_contact > 0.01 and mixsteps <= 0):
            if delta_str == 'mix':
                delta_str = 'sd'  # If the last step was mixing, this is now steepest descent

            # Compute gap and adjust offset (if at constant external force)
            g_r = u_r[comp_mask] - masked_surface
            if external_force is not None:
                offset = 0
                if A_cg > 0:
                    offset = reduction.sum(g_r[c_r[comp_mask]]) / A_cg
            g_r -= offset ## 21a
            #print(g_r)
            
            # Compute G = sum(g*g) (over contact area only)
            G = reduction.sum(c_r[comp_mask] * g_r * g_r) ## 22
    
            # t = (g + delta*(G/G_old)*t) inside contact area and 0 outside
            if delta > 0 and G_old > 0:
                t_r[comp_mask] = c_r[comp_mask] * (g_r + delta * (G / G_old) * t_r[comp_mask]) ## 23
            else:
                t_r[comp_mask] = c_r[comp_mask] * g_r

            # Compute elastic displacement that belong to t_r
            # substrate (Nelastic manifold: r_r is negative of Polonsky,  Kerr's r)
            # r_r = -np.fft.ifft2(gf_q*np.fft.fft2(t_r)).real
            r_r = substrate.evaluate_disp(t_r) ## 24
            result.nfev += 1
            # Note: Sign reversed from Polonsky, Keer because this r_r is negative of theirs.
            tau = 0.0
            if A_cg > 0:
                # tau = -sum(g*t)/sum(r*t) where sum is only over contact region
                x = -reduction.sum(c_r * r_r * t_r)
                if x > 0.0:
                    tau = reduction.sum(c_r[comp_mask] * g_r * t_r[comp_mask]) / x  ## 26
                else:
                    G = 0.0

            f_r += tau * c_r * t_r ## 30

            # Reset mixing factor
            current_mixfac = mixfac
        
        # Find area with tensile stress and negative gap
        # (i.e. penetration of the two surfaces)
        mask_tensile = f_r >= 0.0
        nc_r = np.logical_and(mask_tensile[comp_mask], g_r < 0.0)


        # For nonperiodic calculations: Find maximum force in pad region.
        # This must be zero.
        pad_pres = 0
        if N_pad > 0:
            pad_pres = reduction.max(abs(f_r[pad_mask]))

        # Find maximum force outside contacting region and the deviation
        # from hardness inside the flowing regions. This should go to zero.
        max_pres = 0
        if reduction.sum(mask_tensile * 1) > 0:
            max_pres = reduction.max(f_r[mask_tensile] * 1)

        # Set all tensile stresses to zero
        f_r[mask_tensile] = 0.0

        # Adjust force
        if external_force is not None:
            total_force = -reduction.sum(f_r[comp_mask])
            if total_force != 0:
                f_r *= external_force / total_force ## 31a/b
            else:
                f_r = -external_force / nb_surface_pts * np.ones_like(f_r)
                f_r[pad_mask] = 0.0 # set [16:,16:] = 0


        if delta_str == 'mix':
            delta = 0
            G = 0
        else:
            if reduction.sum(nc_r * 1) > 0:
                # The contact area has changed! nc_r contains area that
                # penetrate but have zero (or tensile) force. They hence
                # violate the contact constraint. Update their forces and
                # reset the CG iteration.
                f_r[comp_mask] += tau * nc_r * g_r
                delta = 0
                delta_str = 'sd'
            else:
                delta = 1
                delta_str = 'cg'

        # Compute new displacements from updated forces
        # u_r = -np.fft.ifft2(gf_q*np.fft.fft2(f_r)).real
        new_u_r = substrate.evaluate_disp(f_r)
        maxdu = reduction.max(abs(new_u_r - u_r))
        u_r = new_u_r
        result.nfev += 1

        # Store G for next step
        G_old = G

        # Compute root-mean square penetration, max penetration and max force
        # difference between the steps
        if A_cg > 0:
            rms_pen = sqrt(G / A_cg)
        else:
            rms_pen = sqrt(G)
        max_pen = max(0.0, reduction.max(c_r[comp_mask] * (masked_surface + offset - u_r[comp_mask])))
        result.maxcv = {"max_pen": max_pen,
                        "max_pres": max_pres}

        # Elastic energy would be
        # e_el = -0.5*reduction.sum(f_r*u_r)

        # Check for change in total force (only at constant offset)
        converged = True
        total_force = -reduction.sum(f_r[comp_mask])
        if external_force is not None:
            converged = converged and abs((total_force - external_force) / total_force) < thermotol
        elif last_total_force is not None:
            converged = converged and abs((total_force - last_total_force) / total_force) < thermotol
        last_total_force = total_force

        # Check for movement of rigid surface (only at constant force)
        max_height = max_masked_surface + offset
        if external_force is not None and last_max_height is not None:
            converged = converged and abs((max_height - last_max_height) / max_height) < thermotol
        last_max_height = max_height

        if delta_str == 'mix':
            # Don't check for penetration as the rigid surface penetrates where the contact is plastic
            converged = converged and maxdu < pentol and max_pres < forcetol and pad_pres < forcetol
        else:
            converged = converged and rms_pen < pentol and max_pen < pentol and maxdu < pentol and \
                        max_pres < forcetol and pad_pres < forcetol


        if converged:
            # Return full u_r because this is required to reproduce force
            # from evaluate_force
            result.x = u_r  # [comp_mask]
            # Return partial f_r because force outside computational region
            # is zero anyway
            result.jac = -f_r[tuple(comp_slice)]
            print(result.jac)
            print(it)
            result.active_set = c_r
            if hardness is not None:
                plastic = np.zeros_like(c_r)
                plastic[comp_mask] = g_r < 0.0
                result.plastic = plastic[tuple(comp_slice)]
            if delta_str == 'mix':
                result.active_set[comp_mask] = g_r < 0.0
            # Compute elastic energy
            result.fun = -reduction.sum(f_r[tuple(comp_slice)] * u_r[tuple(comp_slice)]) / 2
            result.offset = offset
            result.success = True
            result.message = "Polonsky converged"
            return result

        if it < maxiter:
            #logger.st(log_headers, log_values)
            pass
        if callback is not None:
            d = dict(area=np.int64(A_contact).item(),
                     fractional_area=np.float64(
                         A_contact / reduction.sum(surf_mask)).item(),
                     rms_penetration=np.float64(rms_pen).item(),
                     max_penetration=np.float64(max_pen).item(),
                     max_pressure=np.float64(max_pres).item(),
                     pad_pressure=np.float64(pad_pres).item(),
                     penetration_tol=np.float64(pentol).item(),
                     pressure_tol=np.float64(forcetol).item())
            callback(it, f_r, d)

        if isnan(G) or isnan(rms_pen):
            raise RuntimeError('nan encountered.')

    #log_values[0] = 'NOT CONVERGED'
    #logger.st(log_headers, log_values, force_print=True)

    # Return full u_r because this is required to reproduce force
    # from evalualte_force
    result.x = u_r  # [comp_mask]
    # Return partial f_r because force outside computational region
    # is zero anyway
    result.jac = -f_r[tuple(comp_slice)]
    result.active_set = c_r
    # Compute elastic energy
    result.fun = -reduction.sum(
        (f_r[tuple(comp_slice)] * u_r[tuple(comp_slice)])) / 2
    result.offset = offset
    result.message = "Reached maxiter = {}".format(maxiter)
    return result

In [4]:
from collections import namedtuple

import numpy as np

from SurfaceTopography.Support import doi

from ContactMechanics import ElasticSubstrate

from muFFT import FFT
from NuMPI.Tools import Reduction


class PeriodicFFTElasticHalfSpace(ElasticSubstrate):
    """ Uses the FFT to solve the displacements and stresses in an elastic
        Halfspace due to a given array of point forces. This halfspace
        implementation cheats somewhat: since a net pressure would result in
        infinite displacement, the first term of the FFT is systematically
        dropped.
        The implementation follows the description in Stanley & Kato J. Tribol.
        119(3), 481-485 (Jul 01, 1997)
    """

    name = "periodic_fft_elastic_halfspace"
    _periodic = True

    @doi('10.1115/1.2833523',  # Stanley & Kato
         '10.1103/PhysRevB.74.075420',  # Campana & Müser
         '10.1103/PhysRevB.86.075459'  # Pastewka, Sharp, Robbins
         )
    def __init__(self, nb_grid_pts, young, physical_sizes=2 * np.pi,
                 stiffness_q0=None, thickness=None, poisson=0.0,
                 superclass=True, fft="serial", communicator=None):
        """
        Parameters
        ----------
        nb_grid_pts : int tuple
            containing number of points in spatial directions.
            The length of the tuple determines the spatial dimension
            of the problem.
        young : float
            Young's modulus, if poisson is not specified it is the
            contact modulus as defined in Johnson, Contact Mechanics
        physical_sizes : float or float tuple
            (default 2π) domain size.
            For multidimensional problems,
            a tuple can be provided to specify the lengths per
            dimension. If the tuple has less entries than dimensions,
            the last value in repeated.
        stiffness_q0 : float, optional
            Substrate stiffness at the Gamma-point (wavevector q=0).
            If None, this is taken equal to the lowest nonvanishing
            stiffness. Cannot be used in combination with thickness.
        thickness : float, optional
            Thickness of the elastic half-space. If None, this
            models an infinitely deep half-space. Cannot be used in
            combination with stiffness_q0.
        poisson : float
            Default 0
             Poisson number. Need only be specified for substrates
             of finite thickness. If left unspecified for substrates
             of infinite thickness, then young is the contact
             modulus.
        superclass : bool
            (default True)
            client software never uses this.
            Only inheriting subclasses use this.
        fft: string
            Default: 'serial'
            FFT engine to use. Options are 'fftw', 'fftwmpi', 'pfft' and
            'p3dfft'. 'serial' and 'mpi' can also be specified, where the
            choice of the appropriate fft is made by muFFT
        communicator : mpi4py communicator or NuMPI stub communicator
            MPI communicator object.
        """
        super().__init__()
        if not hasattr(nb_grid_pts, "__iter__"):
            nb_grid_pts = (nb_grid_pts,)
        if not hasattr(physical_sizes, "__iter__"):
            physical_sizes = (physical_sizes,)
        self.__dim = len(nb_grid_pts)
        if self.dim not in (1, 2):
            raise self.Error(
                ("Dimension of this problem is {}. Only 1 and 2-dimensional "
                 "problems are supported").format(self.dim))
        if stiffness_q0 is not None and thickness is not None:
            raise self.Error("Please specify either stiffness_q0 or thickness "
                             "or neither.")
        self._nb_grid_pts = nb_grid_pts
        tmpsize = list()
        for i in range(self.dim):
            tmpsize.append(physical_sizes[min(i, len(physical_sizes) - 1)])
        self._physical_sizes = tuple(tmpsize)

        try:
            self._steps = tuple(
                float(size) / res for size, res in
                zip(self.physical_sizes, self.nb_grid_pts))
        except ZeroDivisionError as err:
            raise ZeroDivisionError(
                ("{}, when trying to handle "
                 "    self._steps = tuple("
                 "        float(physical_sizes)/res for physical_sizes, res in"
                 "        zip(self.physical_sizes, self.nb_grid_pts))"
                 "Parameters: self.physical_sizes = {}, self.nb_grid_pts = {}"
                 "").format(err, self.physical_sizes, self.nb_grid_pts))
        self.young = young
        self.poisson = poisson
        self.contact_modulus = young / (1 - poisson ** 2)
        self.stiffness_q0 = stiffness_q0
        self.thickness = thickness

        self.fftengine = FFT(self.nb_domain_grid_pts, fft=fft,
                             communicator=communicator,
                             allow_temporary_buffer=False,
                             allow_destroy_input=True)
        # Allocate buffers and create plan for one degree of freedom
        self.real_buffer = self.fftengine.register_real_space_field(
            "real-space", 1)
        self.fourier_buffer = self.fftengine.register_fourier_space_field(
            "fourier-space", 1)

        self.greens_function = None
        self.surface_stiffness = None

        self._communicator = communicator
        self.pnp = Reduction(communicator)

        if superclass:
            self.greens_function = self._compute_greens_function()
            self.surface_stiffness = self._compute_surface_stiffness()

    @property
    def dim(self, ):
        "return the substrate's physical dimension"
        return self.__dim

    @property
    def nb_grid_pts(self):
        return self._nb_grid_pts

    @property
    def area_per_pt(self):
        print(self.physical_sizes, self.nb_grid_pts)
        return np.prod(self.physical_sizes) / np.prod(self.nb_grid_pts)

    @property
    def physical_sizes(self):
        return self._physical_sizes

    @property
    def nb_domain_grid_pts(self, ):
        """
        usually, the nb_grid_pts of the system is equal to the geometric
        nb_grid_pts (of the surface). For example free boundary conditions,
        require the computational nb_grid_pts to differ from the geometric one,
        see FreeFFTElasticHalfSpace.
        """
        return self.nb_grid_pts

    @property
    def nb_subdomain_grid_pts(self):
        """
        When working in Parallel one processor holds only Part of the Data

        :return:
        """
        return self.fftengine.nb_subdomain_grid_pts

    @property
    def topography_nb_subdomain_grid_pts(self):
        return self.nb_subdomain_grid_pts

    @property
    def subdomain_locations(self):
        """
        When working in Parallel one processor holds only Part of the Data

        :return:
        """
        return self.fftengine.subdomain_locations

    @property
    def topography_subdomain_locations(self):
        return self.subdomain_locations

    @property
    def subdomain_slices(self):
        """
        When working in Parallel one processor holds only Part of the Data

        :return:
        """
        return self.fftengine.subdomain_slices

    @property
    def topography_subdomain_slices(self):
        return tuple([slice(s, s + n) for s, n in
                      zip(self.topography_subdomain_locations,
                          self.topography_nb_subdomain_grid_pts)])

    @property
    def local_topography_subdomain_slices(self):
        """
        slice representing the local subdomain without the padding area
        """
        return tuple([slice(0, n)
                      for n in self.topography_nb_subdomain_grid_pts])

    @property
    def nb_fourier_grid_pts(self):
        """
        When working in Parallel one processor holds only Part of the Data

        :return:
        """
        return self.fftengine.nb_fourier_grid_pts

    @property
    def fourier_locations(self):
        """
        When working in Parallel one processor holds only Part of the Data

        :return:
        """
        return self.fftengine.fourier_locations

    @property
    def fourier_slices(self):
        """
        When working in Parallel one processor holds only Part of the Data

        :return:
        """
        return self.fftengine.fourier_slices

    @property
    def communicator(self):
        """Return the MPI communicator"""
        return self._communicator

    def __repr__(self):
        dims = 'x', 'y', 'z'
        size_str = ', '.join('{}: {}({})'.format(dim, size, nb_grid_pts) for
                             dim, size, nb_grid_pts in
                             zip(dims, self.physical_sizes, self.nb_grid_pts))
        return "{0.dim}-dimensional halfspace '{0.name}', " \
               "physical_sizes(nb_grid_pts) in {1}, E' = {0.young}" \
            .format(self, size_str)

    def _compute_greens_function(self):
        r"""
        Compute the weights w relating fft(displacement) to fft(pressure):
        fft(u) = w*fft(p), see (6) Stanley & Kato J. Tribol. 119(3), 481-485
        (Jul 01, 1997).

        For the infinite halfspace,
        .. math ::

            w = q E^* / 2

        q is the wavevector (:math:`2 \pi / wavelength`)

        WARNING: the paper is dimensionally *incorrect*. see for the correct
        1D formulation: Section 13.2 in
            K. L. Johnson. (1985). Contact Mechanics. [Online]. Cambridge:
            Cambridge  University Press. Available from: Cambridge Books Online
            <http://dx.doi.org/10.1017/CBO9781139171731> [Accessed 16 February
            2015]
        for correct 2D formulation: Appendix 1, eq A.2 in
            Johnson, Greenwood and Higginson, "The Contact of Elastic Regular
            Wavy surfaces", Int. J. Mech. Sci. Vol. 27 No. 6, pp. 383-396, 1985
            <http://dx.doi.org/10.1016/0020-7403(85)90029-3> [Accessed 18 March
            2015]
        """
        if self.dim == 1:
            nx, = self.nb_grid_pts
            sx, = self.physical_sizes
            # Note: q-values from 0 to 1, not from 0 to 2*pi
            qx = np.arange(self.fourier_locations[0],
                           self.fourier_locations[0] +
                           self.nb_fourier_grid_pts[0], dtype=np.float64)
            qx = np.where(qx <= nx // 2, qx / sx, (nx - qx) / sx)
            surface_stiffness = np.pi * self.contact_modulus * qx

            if self.stiffness_q0 is None:
                surface_stiffness[0] = surface_stiffness[1].real
            elif self.stiffness_q0 == 0.0:
                surface_stiffness[0] = 1.0
            else:
                surface_stiffness[0] = self.stiffness_q0

            greens_function = 1 / surface_stiffness
            if self.fourier_locations == (0,):
                if self.stiffness_q0 == 0.0:
                    greens_function[0, 0] = 0.0

        elif self.dim == 2:
            if np.prod(self.nb_fourier_grid_pts) == 0:
                greens_function = np.zeros(self.nb_fourier_grid_pts, order='f',
                                           dtype=complex)
            else:
                nx, ny = self.nb_grid_pts
                sx, sy = self.physical_sizes
                # Note: q-values from 0 to 1, not from 0 to 2*pi
                qx = np.arange(self.fourier_locations[0],
                               self.fourier_locations[0] +
                               self.nb_fourier_grid_pts[0], dtype=np.float64)
                qx = np.where(qx <= nx // 2, qx / sx, (nx - qx) / sx)
                qy = np.arange(self.fourier_locations[1],
                               self.fourier_locations[1] +
                               self.nb_fourier_grid_pts[1], dtype=np.float64)
                qy = np.where(qy <= ny // 2, qy / sy, (ny - qy) / sy)
                q = np.sqrt((qx * qx).reshape(-1, 1) +
                            (qy * qy).reshape(1, -1))
                if self.fourier_locations == (0, 0):
                    q[0, 0] = np.NaN
                    # q[0,0] has no Impact on the end result,
                    # but q[0,0] =  0 produces runtime Warnings
                    # (because corr[0,0]=inf)
                surface_stiffness = np.pi * self.contact_modulus * q
                #                   E* / 2 (2 \pi / \lambda)
                #                   (q is 1 / lambda, here)
                if self.thickness is not None:
                    # Compute correction for finite thickness
                    q *= 2 * np.pi * self.thickness
                    fac = 3 - 4 * self.poisson
                    off = 4 * self.poisson * (2 * self.poisson - 3) + 5
                    with np.errstate(over="ignore", invalid="ignore",
                                     divide="ignore"):
                        corr = (fac * np.cosh(2 * q) + 2 * q ** 2 + off) / \
                               (fac * np.sinh(2 * q) - 2 * q)
                    # The expression easily overflows numerically. These are
                    # then q-values that are converged to the infinite system
                    # expression.
                    corr[np.isnan(corr)] = 1.0
                    surface_stiffness *= corr
                    if self.fourier_locations == (0, 0):
                        surface_stiffness[0, 0] = \
                            self.young / self.thickness * \
                            (1 - self.poisson) / ((1 - 2 * self.poisson) *
                                                  (1 + self.poisson))
                else:
                    if self.fourier_locations == (0, 0):
                        if self.stiffness_q0 is None:
                            surface_stiffness[0, 0] = \
                                (surface_stiffness[1, 0].real +
                                 surface_stiffness[0, 1].real) / 2
                        elif self.stiffness_q0 == 0.0:
                            surface_stiffness[0, 0] = 1.0
                        else:
                            surface_stiffness[0, 0] = self.stiffness_q0

                greens_function = 1 / surface_stiffness
                if self.fourier_locations == (0, 0):
                    if self.stiffness_q0 == 0.0:
                        greens_function[0, 0] = 0.0
        return greens_function

    def _compute_surface_stiffness(self):
        """
        Invert the weights w relating fft(displacement) to fft(pressure):
        """
        surface_stiffness = np.zeros(self.nb_fourier_grid_pts, order='f',
                                     dtype=complex)
        surface_stiffness[self.greens_function != 0] = \
            1. / self.greens_function[self.greens_function != 0]
        #print("surf stiffness \n",surface_stiffness.real)
        return surface_stiffness

    def evaluate_disp(self, forces):
        """ Computes the displacement due to a given force array
        Keyword Arguments:
        forces   -- a numpy array containing point forces (*not* pressures)
        """
        if forces.shape != self.nb_subdomain_grid_pts:
            raise self.Error(
                ("force array has a different shape ({0}) than this "
                 "halfspace's nb_grid_pts ({1})").format(
                    forces.shape, self.nb_subdomain_grid_pts))
        self.real_buffer.array()[...] = -forces
        self.fftengine.fft(self.real_buffer, self.fourier_buffer)
        self.fourier_buffer.array()[...] *= self.greens_function
        self.fftengine.ifft(self.fourier_buffer, self.real_buffer)
        return self.real_buffer.array().real / \
            self.area_per_pt * self.fftengine.normalisation

    def evaluate_force(self, disp):
        """ Computes the force (*not* pressures) due to a given displacement
        array.

        Keyword Arguments:
        disp   -- a numpy array containing point displacements
        """
        if disp.shape != self.nb_subdomain_grid_pts:
            raise self.Error(
                ("displacements array has a different shape ({0}) than "
                 "this halfspace's nb_grid_pts ({1})").format(
                    disp.shape, self.nb_subdomain_grid_pts))
        self.real_buffer.array()[...] = disp
        self.fftengine.fft(self.real_buffer, self.fourier_buffer)
        self.fourier_buffer.array()[...] *= self.surface_stiffness
        # print('surfstiff\n',self.surface_stiffness)
        self.fftengine.ifft(self.fourier_buffer, self.real_buffer)
        # print("normalization \n",self.fftengine.normalisation)
        # print('area: ',self.area_per_pt )
        return -self.real_buffer.array().real * \
            self.area_per_pt * self.fftengine.normalisation

    def evaluate_k_disp(self, forces):
        """ Computes the K-space displacement due to a given force array

        Parameters
        __________

        forces : ndarray
            a numpy array containing point forces (*not* pressures)

        Returns
        _______

        displacement  :  ndarray
                        displacement in k-space
        """
        if forces.shape != self.nb_subdomain_grid_pts:
            raise self.Error(
                ("force array has a different shape ({0}) than this halfspace'"
                 "s nb_grid_pts ({1})").format(
                    forces.shape, self.nb_subdomain_grid_pts))  # nopep8
        self.real_buffer.array()[...] = -forces
        self.fftengine.fft(self.real_buffer, self.fourier_buffer)
        return self.greens_function * \
            self.fourier_buffer.array() / self.area_per_pt

    def evaluate_k_force(self, disp):
        """ Computes the K-space forces (*not* pressures) due to a given
        displacement array.

        Keyword Arguments:
        disp   -- a numpy array containing point displacements
        """
        if disp.shape != self.nb_subdomain_grid_pts:
            raise self.Error(
                ("displacements array has a different shape ({0}) than this "
                 "halfspace's nb_grid_pts ({1})").format(
                    disp.shape, self.nb_subdomain_grid_pts))  # nopep8
        self.real_buffer.array()[...] = disp
        self.fftengine.fft(self.real_buffer, self.fourier_buffer)
        return -self.surface_stiffness * self.fourier_buffer.array() * \
            self.area_per_pt

    def evaluate_k_force_k(self, disp_k):
        """ Computes the K-space forces (*not* pressures) due to a given
        K-space displacement array.

        Parameters
        __________

        disp : ndarray k-space
            a numpy k-space array containing point displacements

        Returns
        _______

        force_k : nd array k-sapce forces

        """

        return -self.surface_stiffness * disp_k * self.area_per_pt

    def evaluate_elastic_energy(self, forces, disp):
        """
        computes and returns the elastic energy due to forces and displacements
        Arguments:
        forces -- array of forces
        disp   -- array of displacements
        """
        # pylint: disable=no-self-use
        return .5 * self.pnp.dot(np.ravel(disp), np.ravel(-forces))

    def evaluate_scalar_product_k_space(self, ka, kb):
        r"""
        Computes the scalar product, i.e. the power, between the `a` and `b`,
        given their fourier representation.

        `Power theorem
        <https://ccrma.stanford.edu/~jos/mdft/Power_Theorem.html>`_:

        .. math ::

            P = \sum_{ij} a_{ij} b_{ij} =
                \frac{1}{n_x n_y}\sum_{ij}
                \tilde a_{ij} \overline{\tilde b_{ij}}

        Note that for `a`, `b` real,

        .. math :: P = \sum_{kl} Re(\tilde a_{kl}) Re(\tilde b_{kl})
        + Im(\tilde a_{kl}) Im(\tilde b_{kl})


        Parameters
        ----------
        ka, kb:
            arrays of complex type and of size substrate.nb_fourier_grid_pts
            Fourier representation (output of a 2D rfftn) `a` (resp. `b`)
            (`nx, ny` real array)


        Returns
        -------
        P
            The scalar product of a and b

        """

        # ka and kb are the output of the 2D rfftn, that means the a
        # part of the transform is omitted because of the symetry along the
        # last dimension
        #
        # That's why the components whose symetrics have been omitted are
        # weighted with a factor of 2.
        #
        # The first column (indexes [...,0], wavevector 0 along the last
        # dimension) has no symetric
        #
        # When the number of points in the last dimension is even, the last
        # column (Nyquist Frequency) has also no symetric.
        #
        # The serial code implementation would look like this
        # if (self.nb_domain_grid_pts[-1] % 2 == 0)
        #   return .5*(np.vdot(ka, kb).real +
        #           # adding the data that has been omitted by rfftn
        #           np.vdot(ka[..., 1:-1], kb[..., 1:-1]).real
        #           # because of symetry
        #           )/self.nb_pts
        # else :
        #   return .5 * (np.vdot(ka, kb).real +
        #                  # adding the data that has been omitted by rfftn
        #      #           np.vdot(ka[..., 1:], kb[..., 1:]).real
        #      #           # because of symetry
        #      #           )/self.nb_pts
        #
        # Parallelized Version
        # The inner part of the fourier data should always be symetrized (i.e.
        # multiplied by 2). When the fourier subdomain contains boundary values
        # (wavevector 0 (even and odd) and ny//2 (only for odd)) these values
        # should only be added once

        if ka.size > 0:
            if self.fourier_locations[0] == 0:
                # First row of this fourier data is first of global data
                fact0 = 1
            elif self.nb_fourier_grid_pts[0] > 1:
                # local first row is not the first in the global data
                fact0 = 2
            else:
                fact0 = 0

            if self.fourier_locations[0] == 0 and \
                    self.nb_fourier_grid_pts[0] == 1:
                factend = 0
            elif (self.nb_domain_grid_pts[0] % 2 == 1):
                # odd number of points, last row have always to be symmetrized
                factend = 2
            elif self.fourier_locations[0] + \
                    self.nb_fourier_grid_pts[0] - 1 == \
                    self.nb_domain_grid_pts[0] // 2:
                # last row of the global rfftn already contains it's symmetric
                factend = 1
                # print("last Element of the even data has to be accounted
                # only once")
            else:
                factend = 2
                # print("last element of this local slice is not last element
                # of the total global data")
            # print("fact0={}".format(fact0))
            # print("factend={}".format(factend))

            if self.nb_fourier_grid_pts[0] > 2:
                factmiddle = 2
            else:
                factmiddle = 0

            # vdot(a, b) = conj(a) .  b
            locsum = (
                    factmiddle * np.vdot(ka[1:-1, ...],
                                         kb[1:-1, ...]).real
                    + fact0 * np.vdot(ka[0, ...], kb[0, ...]).real
                    + factend * np.vdot(ka[-1, ...], kb[-1, ...]).real
            ) / np.prod(self.nb_domain_grid_pts)  # nopep8
            # We divide by the total number of points to get the appropriate
            # normalisation of the Fourier transform (in numpy the division by
            # happens only at the inverse transform)
        else:
            # This handles the case where the processor holds an empty
            # subdomain
            locsum = np.array([], dtype=ka.real.dtype)
        # print(locsum)
        return self.pnp.sum(locsum)

    def evaluate_elastic_energy_k_space(self, kforces, kdisp):
        r"""
        Computes the Energy due to forces and displacements using their Fourier
        representation.

        .. math ::
        
            E_{el} &= - \frac{1}{2} \sum_{ij} u_{ij} f_{ij}  

                   &= - \frac{1}{2} \frac{1}{n_x n_y} \sum_{kl} \tilde u_{kl} \overline{\tilde f_{kl}} 
        (:math:`\tilde f_{ij} = - \tilde K_{ijkl} \tilde u_{kl}`)
        
        In a parallelized code kforces and kdisp contain only the slice 
        attributed to this processor
        
        
        Parameters
        ----------
        kforces: 
            array of complex type and of size substrate.nb_fourier_grid_pts
            Fourier representation (output of a 2D rfftn) of the forces acting on the grid points
        kdisp: 
            array of complex type and of physical_sizes substrate.nb_fourier_grid_pts
            Fourier representation (output of a 2D rfftn) of the displacements of the grid points


        Returns
        -------
        E
            The elastic energy due to the forces and displacements
        """  # noqa: E501, W291, W293

        return - 0.5 * self.evaluate_scalar_product_k_space(kdisp, kforces)

    def evaluate(self, disp, pot=True, forces=False):
        """Evaluates the elastic energy and the point forces
        Keyword Arguments:
        disp   -- array of distances
        pot    -- (default True) if true, returns potential energy
        forces -- (default False) if true, returns forces
        """
        force = potential = None
        if forces:
            force = self.evaluate_force(disp)
            if pot:
                potential = self.evaluate_elastic_energy(force, disp)
        elif pot:
            # kforce = self.evaluate_k_force(disp)
            # TODO: OPTIMISATION: here kdisp is computed twice, because it's
            #  needed in kforce
            self.real_buffer.array()[...] = disp
            self.fftengine.fft(self.real_buffer, self.fourier_buffer)
            dispk = self.fourier_buffer.array()[...]
            kforce = self.evaluate_k_force_k(dispk)
            potential = self.evaluate_elastic_energy_k_space(kforce, dispk)
        return potential, force

    def evaluate_k(self, disp_k, pot=True, forces=False):
        """Evaluates the elastic energy and the point forces in fourier sapce
        or k-space or reciprocal space.

        Parameters:
        -----------
        disp_k:
            array of displacements in fourier space
        pot: bool
            (default True) if true, returns potential energy
        forces: bool
            (default False) if true, returns forces
        """
        force_k = potential = None
        if forces:
            force_k = self.evaluate_k_force_k(disp_k)
            if pot:
                potential = self.evaluate_elastic_energy_k_space(force_k,
                                                                 disp_k)
        elif pot:
            force_k = self.evaluate_k_force_k(disp_k)
            potential = self.evaluate_elastic_energy_k_space(force_k, disp_k)
        return potential, force_k


class FreeFFTElasticHalfSpace(PeriodicFFTElasticHalfSpace):
    """
    Uses the FFT to solve the displacements and stresses in an non-periodic
    elastic Halfspace due to a given array of point forces. Uses the Green's
    functions formulaiton of Johnson (1985, p. 54). The application of the FFT
    to a nonperiodic domain is explained in Hockney (1969, p. 178.)

    K. L. Johnson. (1985). Contact Mechanics. [Online]. Cambridge: Cambridge
    University Press. Available from: Cambridge Books Online
    <http://dx.doi.org/10.1017/CBO9781139171731> [Accessed 16 February 2015]

    R. W. HOCKNEY, "The potential calculation and some applications," Methods
    of Computational Physics, B. Adler, S. Fernback and M. Rotenberg (Eds.),
    Academic Press, New York, 1969, pp. 136-211.
    """
    name = "free_fft_elastic_halfspace"
    _periodic = False

    @doi('Hockney, Methods Comput. Phys. 9, 135 (1970)',  # Hockney 1970
         '10.1016/S0043-1648(00)00427-0',  # Liu, Wang, Liu 2000
         '10.1063/1.4950802'  # Pastewka & Robbins 2016
         )
    def __init__(self, nb_grid_pts, young, physical_sizes=2 * np.pi,
                 fft="serial", communicator=None, check_boundaries=False):
        """
        Parameters
        ----------
        nb_grid_pts : tuple of floats
            Tuple containing number of points in spatial directions. The length
            of the tuple determines the spatial dimension of the problem.
            Warning: internally, the free boundary conditions require the
            system so store a system of 2*nb_grid_pts.x by 2*nb_grid_pts.y.
            Keep in mind that if your surface is nx by ny, the forces and
            displacements will still be 2nx by 2ny.
        young : float
            Equiv. Young's modulus E', 1/E' = (i-ν_1**2)/E'_1 + (i-ν_2**2)/E'_2
        physical_sizes : tuple of floats
            (default 2π) domain physical_sizes. For multidimensional problems,
            a tuple can be provided to specify the lengths per dimension. If
            the tuple has less entries than dimensions, the last value in
            repeated.
        communicator : mpi4py communicator NuMPI stub communicator
            MPI communicator object.
        check_boundaries: bool
        if set to true, the function check will test that the pressures are
        zero at the boundary of the topography-domain.
        `check()` is called systematically at the end of system.minimize_proxy
        """
        self._comp_nb_grid_pts = tuple((2 * r for r in nb_grid_pts))
        super().__init__(nb_grid_pts, young, physical_sizes, superclass=False,
                         fft=fft, communicator=communicator)
        self.greens_function = self._compute_greens_function()
        self.surface_stiffness = self._compute_surface_stiffness()
        self._check_boundaries = check_boundaries

    def spawn_child(self, nb_grid_pts):
        """
        returns an instance with same physical properties with a smaller
        computational grid
        """
        size = tuple((nb_grid_pts[i] / float(self.nb_grid_pts[i])
                      * self.physical_sizes[i] for i in range(self.dim)))
        return type(self)(nb_grid_pts, self.young, size)

    @property
    def nb_domain_grid_pts(self, ):
        """
        usually, the nb_grid_pts of the system is equal to the geometric
        nb_grid_pts (of the surface). For example free boundary conditions,
        require the computational nb_grid_pts to differ from the geometric one,
        see FreeFFTElasticHalfSpace.
        """
        return self._comp_nb_grid_pts

    @property
    def topography_nb_subdomain_grid_pts(self):
        return tuple([max(0, min(self.nb_grid_pts[i] -
                                 self.subdomain_locations[i],
                                 self.nb_subdomain_grid_pts[i]))
                      for i in range(self.dim)])

    @property
    def domain_boundary_mask(self):
        r"""

        Returns a mask of the points that are on the boundary of the domain

        Returns
        -------
        bool ndarray with size self.topography_nb_subdomain_grid_pts
        """
        mask = np.zeros(self.topography_nb_subdomain_grid_pts, dtype=bool)
        if self.dim == 2:
            if self.subdomain_locations[1] == 0:
                mask[:, 0] = 1

            maxiy = self.nb_grid_pts[1] - 1 - \
                self.topography_subdomain_locations[1]
            if 0 < maxiy < self.topography_nb_subdomain_grid_pts[1]:
                mask[:, maxiy] = 1

            if self.subdomain_locations[0] == 0:
                mask[0, :] = 1

            maxix = self.nb_grid_pts[0] - 1 - \
                self.topography_subdomain_locations[0]
            if 0 < maxix < self.topography_nb_subdomain_grid_pts[0]:
                mask[maxix, :] = 1
        return mask

    def _compute_greens_function(self):
        """Compute the weights w relating fft(displacement) to fft(pressure):
           fft(u) = w*fft(p), Johnson, p. 54, and Hockney, p. 178

           This version is less is copied from matscipy, use if memory is a
           concern
        """
        # pylint: disable=invalid-name
        a = self._steps[0] * .5
        if self.dim == 1:
            pass
        else:
            b = self._steps[1] * .5
            x_s = np.arange(self.subdomain_locations[0],
                            self.subdomain_locations[0] +
                            self.nb_subdomain_grid_pts[0])
            x_s = np.where(x_s <= self.nb_grid_pts[0], x_s,
                           x_s - self.nb_grid_pts[0] * 2) * self._steps[0]
            x_s.shape = (-1, 1)
            y_s = np.arange(self.subdomain_locations[1],
                            self.subdomain_locations[1] +
                            self.nb_subdomain_grid_pts[1])
            y_s = np.where(y_s <= self.nb_grid_pts[1], y_s,
                           y_s - self.nb_grid_pts[1] * 2) * self._steps[1]
            y_s.shape = (1, -1)
            self.real_buffer.array()[...] = 1 / (np.pi * self.young) * (
                    (x_s + a) * np.log(((y_s + b) + np.sqrt((y_s + b) * (y_s + b) +  # noqa: E501
                                                            (x_s + a) * (x_s + a))) /  # noqa: E501
                                       ((y_s - b) + np.sqrt((y_s - b) * (y_s - b) +  # noqa: E501
                                                            (x_s + a) * (x_s + a)))) +  # noqa: E501
                    (y_s + b) * np.log(((x_s + a) + np.sqrt((y_s + b) * (y_s + b) +  # noqa: E501
                                                            (x_s + a) * (x_s + a))) /  # noqa: E501
                                       ((x_s - a) + np.sqrt((y_s + b) * (y_s + b) +  # noqa: E501
                                                            (x_s - a) * (x_s - a)))) +  # noqa: E501
                    (x_s - a) * np.log(((y_s - b) + np.sqrt((y_s - b) * (y_s - b) +  # noqa: E501
                                                            (x_s - a) * (x_s - a))) /  # noqa: E501
                                       ((y_s + b) + np.sqrt((y_s + b) * (y_s + b) +  # noqa: E501
                                                            (x_s - a) * (x_s - a)))) +  # noqa: E501
                    (y_s - b) * np.log(((x_s - a) + np.sqrt((y_s - b) * (y_s - b) +  # noqa: E501
                                                            (x_s - a) * (x_s - a))) /  # noqa: E501
                                       ((x_s + a) + np.sqrt((y_s - b) * (y_s - b) +  # noqa: E501
                                                            (x_s + a) * (x_s + a)))))  # noqa: E501
            self.fftengine.fft(self.real_buffer, self.fourier_buffer)
            return self.fourier_buffer.array().copy()

    def evaluate_disp(self, forces):
        """ Computes the displacement due to a given force array
        Keyword Arguments:
        forces   -- a numpy array containing point forces (*not* pressures)

        if running in MPI this should be only the forces in the Subdomain

        if running in serial one can give the force array with or without the
        padded region

        """
        if forces.shape == self.nb_subdomain_grid_pts:
            return super().evaluate_disp(forces)

        elif self.nb_subdomain_grid_pts == self.nb_domain_grid_pts:
            if forces.shape == self.nb_grid_pts:
                # Automatically pad forces if force array is half of subdomain
                # nb_grid_pts
                padded_forces = np.zeros(self.nb_domain_grid_pts)
                s = [slice(0, forces.shape[i])
                     for i in range(len(forces.shape))]
                padded_forces[s] = forces
                return super().evaluate_disp(padded_forces)[s]
        else:
            raise self.Error("forces should be of subdomain nb_grid_pts when "
                             "using MPI")

        raise self.Error("force array has a different shape ({0}) "
                         "than the subdomain nb_grid_pts ({1}), this "
                         "halfspace's nb_grid_pts ({2}) or "
                         "half of it.".format(forces.shape,
                                              self.nb_subdomain_grid_pts,
                                              self.nb_domain_grid_pts))

        # possible implementation in parallel with adding gather
        # padded_forces = np.zeros(self.nb_domain_grid_pts)
        # s = [slice(0, max(0, min(self.nb_grid_pts[i] -
        # self.subdomain_locations[i], self.nb_subdomain_grid_pts[i])))
        #     for i in range(self.dim)]
        # padded_forces[s] = forces
        # return super().evaluate_disp(padded_forces)[s]

    class FreeBoundaryError(Exception):
        """
        called when the forces overlap into the padding region
        (i.e. the outer ring of the force array equals zero),
        needing an increase of the nb_grid_pts
        """

        def __init__(self, message):
            super().__init__(message)

    def check_boundaries(self, force=None, tol=0):
        """
        Raises an error if the forces are not zero at the boundary of the
        active domain

        Parameters
        ----------
        force

        Returns
        -------

        """

        if force is None:
            force = self.force
        is_ok = True
        if self.dim == 2:
            if np.ma.is_masked(force):
                def check_vals(vals):
                    return (abs(vals) <= tol).all() or vals.mask.all()
            else:
                def check_vals(vals):
                    return (abs(vals) <= tol).all()

            if self.subdomain_locations[1] == 0:
                is_ok &= check_vals(force[:, 0])

            maxiy = self.nb_grid_pts[1] - 1 - \
                self.topography_subdomain_locations[1]
            if 0 < maxiy < self.topography_nb_subdomain_grid_pts[1]:
                is_ok &= check_vals(force[:, maxiy])

            if self.subdomain_locations[0] == 0:
                is_ok &= check_vals(force[0, :])

            maxix = self.nb_grid_pts[0] - 1 - \
                self.topography_subdomain_locations[0]
            if 0 < maxix < self.topography_nb_subdomain_grid_pts[0]:
                is_ok &= check_vals(force[maxix, :])

        is_ok = self.pnp.all(is_ok)

        if not is_ok:
            raise self.FreeBoundaryError(
                "The forces not zero at the boundary of the active domain."
                "This is typically an indication that the contact geometry "
                "exceeds the bounds of the domain. Since this is a nonperiodic"
                "calculation, you may want to increase the size of your "
                "domain. If you are sure that the calculation is correct,"
                " set check_boundary to False")

    def check(self, force=None):
        """
        Checks wether force is still in the value range handled correctly
        Parameters
        ----------
        force

        Returns
        -------

        """
        if self._check_boundaries:
            self.check_boundaries(force)


# convenient container for storing correspondences betwees small and large
# system
BndSet = namedtuple('BndSet', ('large', 'small'))